In [2]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



count of reviews: 159702


In [3]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
#print(doc_clean[:100])

In [4]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)

doc_clean = None
#print(tagged[:100])

In [5]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
#print(noun_phrase_tagged[:100])
#doc_complete = nouns

tagged = None

MemoryError: 

In [ ]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
print(nouns[:100])
doc_complete = nouns

In [ ]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [ ]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

In [ ]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
try:
    mydb.features.drop()
finally:
    result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [ ]:
features=  [ {"feature" : key} for (key,val) in dict_noun_abs_sentiment.items() if val >0.01]
print(features)

In [ ]:
my_collection.insert_many(features)